Spojenie čiastkových datasetov a kontrola.

In [1]:
import pandas as pd
import numpy as np
import pickle

### Spojenie

In [3]:
df = pd.DataFrame()
datestring = '20191118'

for ii in range(1, 185+1):
    with open('./Data/data_'+datestring+'_' + str(ii) + '.pkl', 'rb') as f:
        df0 = pickle.load(f)
    df = df.append(df0, ignore_index=True)

### Kontrola

In [24]:
df.index.size

18469

In [26]:
df.index_konania.unique().size

18469

bez duplicitných záznamov

Skontroluj preskocene záznamy

In [22]:
with open('./Data/log_preskocene_'+datestring+'.txt') as f:
    preskocene = f.readlines()

# zbehnutie kodu ak zlyha ukladanie niektorych datasetov
# with open('./Data/log_preskocene_chyby'+datestring+'.txt') as f:
#    preskocene = preskocene + f.readlines()

In [49]:
# transformuj do formatu index - (priebežný) výsledok konania
def get_index_result(preskoceny_zaznam):
    preskoceny_zaznam_list = preskoceny_zaznam.split(' ')
    index = int(preskoceny_zaznam_list[1])
    vysledok = ' '.join(preskoceny_zaznam_list[3:]).strip('\n')
    
    return (index, vysledok)
preskocene_list = list(map(get_index_result, preskocene))
print(len(preskocene_list))

# poradime si s duplikatmi (opakovany zapis zaznamov, ktore sa kvoli chybe neulozili ale neskor sme ich znova zbehli)
df_preskocene = pd.DataFrame(pd.Series(dict(preskocene_list)))
df_preskocene.index.name = 'index_konania'
df_preskocene.rename(columns={0:'vysledok'}, inplace=True)
print('Unikatnych', df_preskocene.index.size)

1546
Unikatnych 1340


In [50]:
# co tam máme?
df_preskocene['vysledok'].value_counts()

Ukončené prihlasovanie                                    682
Zrušené výberové konanie                                  404
Vyhlásené                                                 229
Zrušené štátnozamestnanecké miesto                         16
Zánik dôvodu na obsadenie štátnozamestnaneckého miesta      9
Name: vysledok, dtype: int64

In [45]:
df_preskocene.index.size

1340

In [51]:
# nacitaj relativne linky ku vsetkym konaniam
with open('./Data/rel_linky_list_'+datestring+'.pkl', 'rb') as f:
    relativne_linky = pickle.load(f)
    
pocet_zaznamov = len(relativne_linky)

In [53]:
print('pocet_zaznamov == s_vysledkom + preskocene' )
print(pocet_zaznamov == (df.index.size + df_preskocene.index.size))

pocet_zaznamov == s_vysledkom + preskocene
True


In [ ]:
# uloz
df.to_excel('./Data/vyberove_konania_20191119.xlsx')
df.to_pickle('./Data/vyberove_konania_20191119.pkl')